# Trabajando con Dataframes

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Dataframes") \
        .master('local[*]') \
        .getOrCreate()

#spark = SparkSession.builder.appName("Dataframes") \
#        .getOrCreate()

In [ ]:
spark

## Creación 

In [ ]:
data = [('Alicia', 25), ('Bob', 40), ("Charlie",35)]

In [ ]:
columns = ['Name', 'Age']

In [ ]:
# El sistema infiere el esquema
df = spark.createDataFrame(data, columns)
type(df)

In [ ]:
df.printSchema()

df.dtypes

In [ ]:
# Le puedo proporcionar el esquema
schema = 'Name: string, Age: int'
df = spark.createDataFrame(data, schema)

In [ ]:
df.printSchema()

## Contenido

In [ ]:
df.show()

In [ ]:
df.take(2)

In [ ]:
df.head()

In [ ]:
df.tail(2)

### Podemos obtener un resumén estadístico del **DataFrame**

In [ ]:
# Resumen estadístico (el método describe regresa un nuevo DataFrame)
df.describe().show()

## Particiones

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
#res = df.repartition(4)
#res.rdd.getNumPartitions()

## Seleccionar columnas

### Seleccionar una columna

In [ ]:
df.Age

In [ ]:
df['Name']

### Crear **DataFrame** a partir de una columna

In [ ]:
df.select('Name').show()

In [ ]:
df.select(df.Age).show()

### Crear **DataFrame** a partir de multiples columnas (observa como escogí el orden de columnas distinto al original)

In [ ]:
df.select('Age', 'Name').show()

### Seleccionar todas las columnas en un solo paso

In [ ]:
df.select('*').show()

### Dado que el resultado de `select` es un nuevo **DataFrame**, podemos obtener una descripción estadística de columnas deseadas

In [ ]:
df.select(df.Age).describe().show()

## Filtrado

#### Utilizando una columna y una condición

In [ ]:
df.filter(df.Age > 30).show()

#### Utilizando un `string` con el nombre de una columna y una condición

In [ ]:
df.filter('Age > 30').show()

#### El statement `where` es un alias para `filter`

In [ ]:
df.where(df['Age'] > 30).show()

#### Podemos emplear los operadores lógicos `and` (`&`), `or` (`|`) y `not` (`~`)

In [ ]:
df.where( (df.Age > 30) & (df.Age < 40) ).show()

In [ ]:
df.where( (df.Age < 30) | (df.Age > 36) ).show()

In [ ]:
df.where( ~( (df.Age < 30) | (df.Age > 36) ) ).show()

In [ ]:
df.where( ~(df.Name == 'Bob') ).show()

#### También podemos utilizar expresiones utilizando la sintaxis de SQL

In [ ]:
df.filter( 'Age > 30 AND Age < 40' ).show()

In [ ]:
df.where( 'Age < 30 OR Age > 36' ).show()

In [ ]:
df.where( 'NOT (Name = "Bob")' ).show()

## Modificar

### Renombrar

In [ ]:
# Renombrar una columna
df.withColumnRenamed('Name', 'name').show()

In [ ]:
# Renombrar multiples columnas
df.withColumnsRenamed({'Name':'name', 'Age':'age'}).show()

### Eliminar

#### Para eliminar una o múltiples columnas

In [ ]:
df.drop("Age").show()

In [ ]:
df.drop('Name', 'Age').show()

### Lidiar con valores `NULL`

In [ ]:
data = [('Gilberto', 22, 'Informatico'), 
        ('Raul', None, 'Ingeniero'), 
        ('Karla', 25, 'Abogada'), 
        ('Rocio', None, None), 
        (None, None, None),
        ('Perla', 10, None)]
schema = 'name: string, age: int, carrera: string'

new_df = spark.createDataFrame(data, schema=schema)
new_df.show()

#### Eliminar renglones con valores nulos (`NULL`)

##### Eliminar aquellos con al menos un valor `NULL`

In [ ]:
new_df.na.drop().show()

##### Eliminar cuando todos los valores sean `NULL`

In [ ]:
new_df.na.drop(how = 'all').show()

##### Eliminar cuando el número de valores no nulos sea menor a un limite deseado; en otras palabras asegurar que contemos con un número mínimo de valores (2 en este caso)

In [ ]:
# Al menos dos valores no nulos
new_df.na.drop(thresh = 2).show()

##### Asegurarse que no haya valores nulos en columnas deseadas

In [ ]:
# Solo los renglones donde name y age no sean nulos
new_df.na.drop(subset = ['name', 'age']).show()

#### Reemplazar valores `NULL`

##### Reemplazar en todas las columnas de tipo `string` con el valor 'unknown'

In [ ]:
new_df.na.fill('unknown').show()

##### Reemplazar en todas las columnas numéricas con `-1`

In [ ]:
new_df.na.fill(-1).show()

##### Reemplazar en las columnas deseadas

In [ ]:
new_df.na.fill({'age':-1, 'carrera':'unknown'}).show()

### Modificar el tipo de datos de columnas

#### Modifiquemos la columna `df.Age` de **integer** a **float**

In [ ]:
# DataFrame original
df.printSchema()
df.show()

In [ ]:
# Nuevo DataFrame con schema deseado
from pyspark.sql.types import FloatType
new_df = df.withColumn('Age', df.Age.cast(FloatType()))

new_df.printSchema()
new_df.show()

### Agregar una columna con un valor constante

In [ ]:
from pyspark.sql.functions import lit

new_df.withColumn("ConstantValue", lit(100)).show()

## Aplicar una función

### Combinar matemáticamente columnas mediante un string con la función deseada

In [ ]:
from pyspark.sql.functions import expr
import numpy as np

In [ ]:
data = [(3.0, 1.0, 0.0), (2.0, 2.0, np.pi/2), (4.0, 0.5, np.pi)]
schema = 'amp:float, omega:float, phase:float'

waves = spark.createDataFrame(data, schema)

In [ ]:
t = 1
exp = f'amp * cos( omega * {t} + phase)'

In [ ]:
exp

### El resultado se puede almacenar en una columna nueva o en una ya existente (en este caso una columna nueva llamada `value`)

In [ ]:
waves.withColumn('value', expr(exp)).show()

### Checando que el resultado sea el correcto

In [ ]:
t = 1
mat = np.array(data)
A = mat[:,0]; w = mat[:,1]; phi = mat[:,2]
A * np.cos(w * t + phi)

## Agrupar y agregación

In [ ]:
df.groupBy?

In [ ]:
df = spark.createDataFrame( [("Roberto", 15), ("Ana", 6), 
                             ("Roberto", 10), ("Ana", 6), 
                             ("Ana", 15)], 
                            'name:string, age:int')
df.show()

### La reducción en el paradigma Mapa-Reducción se puede obtener mediante `groupBy`.

#### Se da primero la llave para que se realize la agrupación y, posteriormente, se llama la función de reducción (la cual aquí se llama agregación)

In [ ]:
df.groupBy('name').count().show()

In [ ]:
df.groupBy('name').sum().show()

### Existen varias operaciones útiles

In [ ]:
df.groupBy('name').avg().show()

df.groupBy('name').max().show()

df.groupBy('name').min().show()

In [ ]:
from pyspark.sql.functions import std

In [ ]:
df.groupby(df.name).agg({'age':'std'}).show()

### Podemos realizar varias agregaciones a la vez

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df.groupby(df.name).agg(F.avg(df.age), F.max(df.age), F.min(df.age)).show()

### La llave es quién yo eliga

In [ ]:
df.groupBy('age').count().show()

### Continuando con la analogía con llaves en Mapa-Reduccion, aquí podemos generar llaves con más de una columna, es decir, agrupar cuando dos columnas toman el mismo valor 

In [ ]:
df.groupBy(['name', 'age']).count().show()

### También podemos llamar a las columnas mediante su número de columna; en este caso ¡las columnas empiezan con el número uno, no el cero!

In [ ]:
df.groupBy(1).sum().show()

df.groupby([1, 2]).count().show()

df.groupBy([df.name, 2]).count().show()

### Si lo preferimos podemos utilizar la sintaxis de **SQL**

#### Primero registramos el **DataFrame** que queremos utilizar con `spark.sql`

In [ ]:
# Registramos una vista temporal del DataFrame con el nombre que deseemos 
# (df_sql en este caso)
df.createOrReplaceTempView("df_sql")

#### Posteriormente podemos seleccionar columnas y aplicar filtros con **SQL**

In [ ]:
spark.sql("SELECT name from df_sql").show()

In [ ]:
spark.sql("SELECT age,name from df_sql").show()

In [ ]:
spark.sql("SELECT * from df_sql").show()

In [ ]:
spark.sql("SELECT * from df_sql WHERE age > 7").show()

#### También podemos aplicar expresiones a columnas empleando la sintaxis de **SQL**

In [ ]:
df.selectExpr("3 * age", "2 * abs(age) as twice_age", "name").show()

In [ ]:
df.selectExpr("name", "age <= 10 as kid").show()

## Ordenando por columnas

In [ ]:
df.orderBy('age').show()

In [ ]:
df.orderBy(df.age.desc()).show()

In [ ]:
df.orderBy('age', 'name').show()

## Aplicaciones de **ML** 

In [ ]:
data = [("Frankestein", 1.5, 0.8, 3.4, 9), 
        ("Alice", 2.9, 4.1, 0.4, 10),
        ("Beowulf", 0.8, 1.7, 2.4, 9),
        ("Moby_Dick", 3.9, 0.1, 2.6, 8),
        ("Romeo_Juliet", 0.9, 2.1, 4.4, 10),
        ("Dorian_Gray", 4.0, 1.2, 2.4, 8),
        ("Pride_Prejudice", 0.5, 0.3, 1.4, 7),
        ("Two_Cities", 5.9, 2.6, 2.1, 9)]

schema = "book: string, v1:float, v2:float, v3:float, value:int"

df_books = spark.createDataFrame(data, schema=schema)
df_books.show()

###  Crear vectores **feature** (mezclar múltiples columnas en un vector)

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
columns_to_mix = ['v1', 'v2', 'v3']
vec_assembler = VectorAssembler(inputCols=columns_to_mix, 
                                outputCol='features')

type(vec_assembler)

In [ ]:
df_with_feat = vec_assembler.transform(df_books)
df_with_feat.show()

### Puedo almacenar la columna en un nuevo **DataFrame** aunque no será necesario para aplicar un modelo de **ML**

In [ ]:
# cada renglon representa un vector feature
X = df_with_feat.select('features')
X.show(truncate=False)

### En **ML** se require dividir aleatoriamente los datos en: datos de entrenamiento (**train set**) y datos de prueba (**test set**). Los **DataFrames** cuentan con una función para lograr esto

In [ ]:
#El seed es opcional 
train_set, test_set = df_with_feat.randomSplit([0.8, 0.2], seed=50)

In [ ]:
train_set.show()
test_set.show()

### Es posible aplicar modelos de **ML** siempre y cuando instalemos el modulo correspondiente ()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
model = LinearRegression(featuresCol='features', labelCol='value')

In [ ]:
trained_model = model.fit(train_set)

In [ ]:
print(model.coefficients)
print(model.intercept)

In [ ]:
model.evaluate(test_set).predictions

In [ ]:
spark.stop()